In [1]:
from src.helper import *
from src.CONSTANTS import *
import matplotlib.pyplot as plt
import wget
import pandas as pd
import numpy as np
import h5py as h5
from random import shuffle
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Dense, GRU, Embedding, Multiply, Attention, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras import backend as k


# Download the dataset
url = 'https://figshare.com/ndownloader/files/12506534'
#wget.download(url)

# Read the downloaded data to a dataframe
with h5.File('D:/MS2/src/holdout_hcd.hdf5', 'r') as f:
  KEY_ARRAY = ["sequence_integer", "precursor_charge_onehot", "intensities_raw"]
  KEY_SCALAR = ["collision_energy_aligned_normed", "collision_energy"]
  df = pd.DataFrame({key: list(f[key][...]) for key in KEY_ARRAY})
  for key in KEY_SCALAR:
    df[key] = f[key][...]

# Add convenience columns
df['precursor_charge'] = df.precursor_charge_onehot.map(lambda a: a.argmax() + 1)
df['sequence_maxquant'] = df.sequence_integer.map(lambda s: "".join(PROSIT_INDEXED_ALPHABET[i] for i in s if i != 0))
df['sequence_length'] = df.sequence_integer.map(lambda s: np.count_nonzero(s))

In [3]:
df['seq1'] = df.sequence_maxquant.map(lambda s: s.replace("M(ox)", "M"))

In [4]:
from pyteomics import mass

df['mass'] = df.seq1.map(lambda s: mass.calculate_mass(s))

In [10]:
df

,sequence_integer,precursor_charge_onehot,intensities_raw,collision_energy_aligned_normed,collision_energy,precursor_charge,sequence_maxquant,sequence_length,seq1,mass
0,"[20, 18, 18, 18, 20, 5, 15, 4, 8, 3, 17, 9, 0,...","[0, 1, 0, 0, 0, 0]","[0.24673085251719526, 0.0, -1.0, 0.0, 0.0, -1....",0.320988,30.0,2,YVVVYFREIDTK,12,YVVVYFREIDTK,1530.808230
1,"[20, 18, 18, 18, 20, 5, 15, 4, 8, 3, 17, 9, 0,...","[0, 0, 1, 0, 0, 0]","[0.15715586535623716, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.296552,28.0,3,YVVVYFREIDTK,12,YVVVYFREIDTK,1530.808230
2,"[20, 18, 18, 18, 20, 5, 15, 4, 8, 3, 17, 9, 0,...","[0, 1, 0, 0, 0, 0]","[0.15716640462180428, 0.0, -1.0, 0.0, 0.0, -1....",0.270988,25.0,2,YVVVYFREIDTK,12,YVVVYFREIDTK,1530.808230
3,"[20, 18, 18, 18, 20, 5, 15, 4, 8, 3, 17, 9, 0,...","[0, 1, 0, 0, 0, 0]","[0.22769115713114144, 0.0, -1.0, 0.0, 0.0, -1....",0.296552,28.0,2,YVVVYFREIDTK,12,YVVVYFREIDTK,1530.808230
4,"[20, 18, 18, 18, 20, 5, 15, 4, 8, 3, 17, 9, 0,...","[0, 1, 0, 0, 0, 0]","[0.31269017980975133, 0.0, -1.0, 0.0, 0.0, -1....",0.370988,35.0,2,YVVVYFREIDTK,12,YVVVYFREIDTK,1530.808230
...,...,...,...,...,...,...,...,...,...,...
754210,"[4, 10, 4, 15, 10, 14, 7, 8, 8, 14, 15, 0, 0, ...","[0, 0, 1, 0, 0, 0]","[0.5543719909959374, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.377262,35.0,3,ELERLQHIIQR,11,ELERLQHIIQR,1433.810296
754211,"[4, 10, 4, 15, 10, 14, 7, 8, 8, 14, 15, 0, 0, ...","[0, 0, 1, 0, 0, 0]","[0.21019231976547664, 0.0, 0.0, 0.004888404334...",0.327262,30.0,3,ELERLQHIIQR,11,ELERLQHIIQR,1433.810296
754212,"[4, 10, 4, 15, 10, 14, 7, 8, 8, 14, 15, 0, 0, ...","[0, 0, 1, 0, 0, 0]","[0.443151289166258, 0.0, 0.0, 0.00918162143295...",0.377262,35.0,3,ELERLQHIIQR,11,ELERLQHIIQR,1433.810296
754213,"[4, 10, 4, 15, 10, 14, 7, 8, 8, 14, 15, 0, 0, ...","[0, 0, 1, 0, 0, 0]","[0.08849367171148294, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.221111,20.0,3,ELERLQHIIQR,11,ELERLQHIIQR,1433.810296


In [13]:
df.to_csv('D:/test1.csv')

KeyboardInterrupt: 